In [18]:
import openai
import pandas as pd
import json
import yaml
from IPython.display import Markdown
import sys
import yaml
from tqdm import tqdm

In [19]:
zz=openai.Model.list()

In [20]:
len(zz['data'])

62

In [21]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [22]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load prompt_template.md

In [23]:
sys.path.append('..')
from utils import *
from functools import partial

In [24]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [25]:
prompt_dict().keys()

dict_keys(['Preamble', 'Problem-solver', 'Conversation summarizer', 'Construct Kinetic Hamiltonian (continuum version, single-particle)', 'Construct Kinetic Hamiltonian (continuum version, second-quantized)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 

# Prompt fill place holder

In [26]:
import json

In [27]:
pwd=os.getcwd()

In [29]:
arxiv_number=pwd.split('/')[-1]

In [30]:
with open(f'{arxiv_number}.yaml','r') as f:
    kwargs_yaml = yaml.safe_load(f)
# with open(f'{arxiv_number}.jsonl','r') as f:
#     kwargs_jsonl= [json.loads(line) for line in f]

FileNotFoundError: [Errno 2] No such file or directory: '2012.04554.yaml'

In [18]:
def load_excerpt(sources):
    excerpt=''
    for tex, lines in sources.items():
        with open(tex,'r') as f:
            f_list=list(f)
            for line in lines:
                excerpt=excerpt+''.join(f_list[line[0]:line[1]])
    return excerpt

In [19]:
prompt_system='''I will provide you a Excerpt of physics paper, and a Template. You will need to fill the placeholders in the template using the correct information from the excerpt.
Here are conventions: 
{..} means a placeholder which you need to fill by extracting information from the excerpt.
{A|B} means you need to make a choice between A and B
[..] means optional sentence. You should decide whether to use it depending on the excerpt.
{{..}} DOES NOT mean a placeholder. You should not change the content inside double curly braces {{..}}.
`You should recall that {expression_second_nonint}.` : this sentence should be kept as is.


Finally, if you cannot figure out the placeholder, you should leave it as is.'''

In [20]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [21]:
def extractor(descriptor,message=False):
    sys_msg=[{'role': 'system', 'content': prompt_system}]
    question_prompt='\nTemplate:\n {template} \n\n Excerpt:\n {excerpt}'.format(template=drop_text_after(prompt_dict()[descriptor['task']]), excerpt=load_excerpt(descriptor['source']))
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    if message:
        return sys_msg[0]['content']+user_msg[0]['content']
    rs= openai.ChatCompletion.create(messages=messages, **model_params)
    response=rs['choices'][0]['message'].content
    return response


## GPT4

In [26]:
string=''
for kwargs in tqdm(kwargs_yaml):
    string+='# '+kwargs['task']+'\n'
    response=(extractor(kwargs, message=False))
    string+=response+'\n\n'
    
    

100%|██████████| 11/11 [05:14<00:00, 28.63s/it]


In [28]:
print(string)

# Construct Kinetic Hamiltonian (continuum version, second-quantized)
You will be instructed to describe the kinetic term of Hamiltonian in a system under an in-plane magnetic field in the momentum space in the single-particle form.   
The degrees of freedom of the system are: four-component spinors $\psi_{\bm k}=(a_{c\uparrow\bm k},a_{v\uparrow\bm k},a_{c\downarrow\bm k},a_{v\downarrow\bm k})^T$, where $c$ and $v$ denote the conduction and valence bands, and $\uparrow$ and $\downarrow$ denote two opposite spins.  
Express the Kinetic Hamiltonian $H_{\rm{BHZ}}$ using $h_{\uparrow}$, $h_{\downarrow}$, $\psi_{\bm k}^{\dagger}$, and $\psi_{\bm k}$, where the summation of ${k}$ should be running over the entire space.

Use the following conventions for the symbols:  
$h_{\uparrow}$ and $h_{\downarrow}$ are the two $2\times 2$ matrices that can be explicitly expressed as given in the excerpt. $m_e$ and $m_h$ are the effective masses of electrons and holes, $E_g$ is the band gap, $A$ is the 

In [30]:
with open(pwd.split('/')[-1]+'_extractor.md','w') as f:
    f.write(string)